<a href="https://colab.research.google.com/github/aayushs7ha/GeoCoding/blob/main/ReverseGeocoding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Install GDAL and Geopandas
!apt install gdal-bin python-gdal python3-gdal --quiet
!apt install python3-rtree --quiet
!pip install git+git://github.com/geopandas/geopandas.git --quiet
!pip install descartes --quiet
!pip install geopy
!pip install plotly_express
!pip install ipython-autotime


Reading package lists...
Building dependency tree...
Reading state information...
Package python-gdal is not available, but is referred to by another package.
This may mean that the package is missing, has been obsoleted, or
is only available from another source
However the following packages replace it:
  gdal-bin

E: Package 'python-gdal' has no installation candidate
Reading package lists...
Building dependency tree...
Reading state information...
The following additional packages will be installed:
  javascript-common libjs-jquery libjs-underscore libspatialindex-c6
  libspatialindex-dev libspatialindex6
Suggested packages:
  apache2 | lighttpd | httpd
The following NEW packages will be installed:
  javascript-common libjs-jquery libjs-underscore libspatialindex-c6
  libspatialindex-dev libspatialindex6 python3-rtree
0 upgraded, 7 newly installed, 0 to remove and 13 not upgraded.
Need to get 829 kB of archives.
After this operation, 3,054 kB of additional disk space will be used.
G

* Restart the run time. tqdm does not work with recent python so we downgrade it to get it working

In [2]:
!pip install tqdm==4.36.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.7/52.7 kB 3.3 MB/s eta 0:00:00
  Attempting uninstall: tqdm
    Found existing installation: tqdm 4.65.0
    Uninstalling tqdm-4.65.0:
      Successfully uninstalled tqdm-4.65.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
panel 0.14.4 requires tqdm>=4.48.0, but you have tqdm 4.36.0 which is incompatible.
prophet 1.1.4 requires tqdm>=4.36.1, but you have tqdm 4.36.0 which is incompatible.
spacy 3.5.3 requires tqdm<5.0.0,>=4.38.0, but you have tqdm 4.36.0 which is incompatible.


In [ ]:
%load_ext autotime
import pandas as pd

import geopandas as gpd
import geopy
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter

import matplotlib.pyplot as plt
import plotly_express as px
import tqdm
from tqdm import tqdm
from tqdm._tqdm_notebook import tqdm_notebook

## Reverse Geocoding Single Example

In [ ]:
locator = Nominatim(user_agent="myGeocoder")
coordinates = "53.480837, -2.244914"
location = locator.reverse(coordinates)

time: 784 ms


In [ ]:
location.raw

{'address': {'building': 'Eagle Insurance Buildings',
  'city': 'Manchester',
  'country': 'United Kingdom',
  'country_code': 'gb',
  'county': 'Greater Manchester',
  'house_number': '68',
  'postcode': 'M2 4JG',
  'road': 'Cross Street',
  'state': 'England',
  'state_district': 'North West England',
  'suburb': 'City Centre'},
 'boundingbox': ['53.480856', '53.4810634', '-2.2451761', '-2.2449576'],
 'display_name': 'Eagle Insurance Buildings, 68, Cross Street, City Centre, Manchester, Greater Manchester, North West England, England, M2 4JG, United Kingdom',
 'lat': '53.4809597',
 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright',
 'lon': '-2.24506682746292',
 'osm_id': 37139875,
 'osm_type': 'way',
 'place_id': 86197794}

time: 8.84 ms


In [ ]:
print(location.address)

Eagle Insurance Buildings, 68, Cross Street, City Centre, Manchester, Greater Manchester, North West England, England, M2 4JG, United Kingdom
time: 1.69 ms


In [ ]:
location.raw["display_name"]

'Eagle Insurance Buildings, 68, Cross Street, City Centre, Manchester, Greater Manchester, North West England, England, M2 4JG, United Kingdom'

time: 4.28 ms


## Reverse Geocoding with Pandas

In [ ]:
url = "https://www.dropbox.com/s/15gisj8hx218rn1/street-pole-sample.csv?dl=1"
cols = ["X", "Y", "POLE_NUM","TYPE","HEIGHT","POLE_DATE","OWNER"]
df = pd.read_csv(url, usecols=cols)
df.head()

,X,Y,POLE_NUM,TYPE,HEIGHT,POLE_DATE,OWNER
0,-75.170097,39.942766,214423,WP,NaN,1997-06-09T00:00:00.000Z,PECO
1,-75.166112,39.941477,215645,AAPT,25.0,1997-06-10T00:00:00.000Z,Streets
2,-75.163483,39.943068,215926,WP,NaN,1997-06-04T00:00:00.000Z,PECO
3,-75.167727,39.944528,214515,WP,25.0,1997-06-09T00:00:00.000Z,Streets
4,-75.171863,39.941486,214380,WP,25.0,1997-06-09T00:00:00.000Z,Streets


time: 965 ms


In [ ]:
df.shape

(150, 7)

time: 2.61 ms


In [ ]:
import plotly_express as px
px.set_mapbox_access_token("pk.eyJ1Ijoic2hha2Fzb20iLCJhIjoiY2plMWg1NGFpMXZ5NjJxbjhlM2ttN3AwbiJ9.RtGYHmreKiyBfHuElgYq_w")
px.scatter_mapbox(df, lat="Y", lon="X",  zoom=15)

time: 905 ms


In [ ]:
df["geom"] =  df["Y"].map(str)  + ',' + df['X'].map(str)
df.head()

,X,Y,POLE_NUM,TYPE,HEIGHT,POLE_DATE,OWNER,geom
0,-75.170097,39.942766,214423,WP,NaN,1997-06-09T00:00:00.000Z,PECO,"39.9427660880249,-75.17009743393821"
1,-75.166112,39.941477,215645,AAPT,25.0,1997-06-10T00:00:00.000Z,Streets,"39.941477314134396,-75.166112027818"
2,-75.163483,39.943068,215926,WP,NaN,1997-06-04T00:00:00.000Z,PECO,"39.9430681055253,-75.1634826347411"
3,-75.167727,39.944528,214515,WP,25.0,1997-06-09T00:00:00.000Z,Streets,"39.9445279530865,-75.16772651013079"
4,-75.171863,39.941486,214380,WP,25.0,1997-06-09T00:00:00.000Z,Streets,"39.9414861490123,-75.17186304554191"


time: 48.8 ms


In [ ]:
df.geom[0]



'39.9427660880249,-75.17009743393821'

time: 8.51 ms


In [ ]:
locator = Nominatim(user_agent="myGeocoder", timeout=10)
rgeocode = RateLimiter(locator.reverse, min_delay_seconds=0.001)
rgeocode

time: 7.06 ms


In [ ]:
tqdm.pandas()
df['address'] = df['geom'].progress_apply(rgeocode)

100%|██████████| 150/150 [01:44<00:00,  1.43it/s]

time: 1min 44s


In [ ]:
df.head()


,X,Y,POLE_NUM,TYPE,HEIGHT,POLE_DATE,OWNER,geom,address
0,-75.170097,39.942766,214423,WP,NaN,1997-06-09T00:00:00.000Z,PECO,"39.9427660880249,-75.17009743393821","(723, South Chadwick Street, Hawthorne, South ..."
1,-75.166112,39.941477,215645,AAPT,25.0,1997-06-10T00:00:00.000Z,Streets,"39.941477314134396,-75.166112027818","(Tindley Temple United Methodist Church, 750, ..."
2,-75.163483,39.943068,215926,WP,NaN,1997-06-04T00:00:00.000Z,PECO,"39.9430681055253,-75.1634826347411","(South 13th Street, Hawthorne, South Philadelp..."
3,-75.167727,39.944528,214515,WP,25.0,1997-06-09T00:00:00.000Z,Streets,"39.9445279530865,-75.16772651013079","(1538, Naudain Street, Rittenhouse Square, Phi..."
4,-75.171863,39.941486,214380,WP,25.0,1997-06-09T00:00:00.000Z,Streets,"39.9414861490123,-75.17186304554191","(Marion Anderson Community Center, Catharine S..."


time: 28 ms


In [ ]:
df["address"][0]

Location(723, South Chadwick Street, Hawthorne, South Philadelphia, Philadelphia, Philadelphia County, Pennsylvania, 19146, United States, (39.9427343888889, -75.1700092777778, 0.0))

time: 3.78 ms
